### This notebook runs simulation with color coding using [node2vec](https://snap.stanford.edu/node2vec/) embedding

In [1]:
import pandas as pd
import numpy as np
import random
from random import shuffle, randrange
from collections import defaultdict
import utilities

import matplotlib.pyplot as plt
import seaborn as sns

import json

import AgentClass
import const
import networkx as nx
from config import num_agents,number_of_bits
from scipy import stats
from collections import defaultdict
import copy

from sklearn.decomposition import TruncatedSVD

In [2]:
%matplotlib inline

In [3]:
random.seed(1)
np.random.seed(1)

In [4]:
def get_tau_distr():
        lower = 0
        upper = 1
        mu = 0.5
        sigma = 0.1
        N = 1000
        
        samples = stats.truncnorm.rvs(
          (lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=N)
        
        return samples

In [5]:
def create_environment():
    list_agents = []
    tau_distr = get_tau_distr()
    
    for i in range(num_agents):
        in_state = [first_bit.pop(), second_bit.pop(), third_bit.pop(), fourth_bit.pop(), fifth_bit.pop()
                   , sixth_bit.pop(), seventh_bit.pop(), eigth_bit.pop(), ninth_bit.pop(), tenth_bit.pop()]
        a = AgentClass.Agent(name='agent{}'.format(i), tau=random.choice(tau_distr), initial_state=in_state)
        list_agents.append(a)
        
    # create network
    G = nx.newman_watts_strogatz_graph(num_agents, 10, 0.5, seed= 0)
#    nx.draw(G, with_labels=True, font_weight='bold') # to draw agents
    df = nx.to_pandas_adjacency(G, dtype=int)
    
    tmp_edges = df.apply(lambda row: row.to_numpy().nonzero()).to_dict()
    edges = {k: v[0].tolist() for k, v in tmp_edges.items()}
    
    # make connections with the agents
    for k, v in edges.items():
        for ngh in v:
            list_agents[k].add_neighbors(list_agents[ngh])
            
    return list_agents

In [6]:
def get_network_df(list_agents):
    network_df = pd.DataFrame({'Agent Name':[], 'Neighbors':[]})
    for agt in list_agents:
        neighbors = agt.get_neighbors_name()
        network_df = network_df.append({'Agent Name':agt.name, 
                                        'Neighbors':neighbors}, ignore_index=True)
    return network_df

Generate coherence matrix for the simulations from 'const.py'

In [7]:
random.seed(1)
  
constants = const.Constants()
coherence_matrix = constants.get_coh_matrix().tolist()


Get the network weight for knowledge transitions using the coherence matrix

First create a dictionary for binary values of each states

In [8]:
kbits_dict = defaultdict(list)

for i in range(1024):
    kbits_dict['State'].append(i)
    kbits_dict['binary'].append('{0:010b}'.format(i))    
    kbits_dict['binary_list'].append(list('{0:010b}'.format(i))[::-1])
kbits_mapper_df = pd.DataFrame(kbits_dict)
kbits_mapper_df.set_index('State', inplace=True)
kbits_mapper_df.drop('binary', inplace=True, axis=1)
kbits_mapper_df_dict = kbits_mapper_df.to_dict(orient='index')

For each state, check every other states and see if bit has to be flipped. If it has to then get the probability value from coherence matrix if it need not then get probability value as 1-(value from coherence matrix).

The total probability would be multiplication of all those.

In [9]:
prob_transitions_mapper = {}

for i in kbits_mapper_df_dict:
    in_state = kbits_mapper_df_dict[i]['binary_list']
    next_state_val = []
    for j in range(1024):
        out_state = kbits_mapper_df_dict[j]['binary_list']
        coh_prob = coherence_matrix[i]
        prob_list = []
        for bit_index in range(len(out_state)):
            if in_state[bit_index] == out_state[bit_index]: # do not change
                prob_bit = 1-coh_prob[bit_index]
            else:
                prob_bit = coh_prob[bit_index]
                
            prob_list.append(prob_bit)
        next_state_val.append(np.prod(np.array(prob_list)))
    prob_transitions_mapper[i] = next_state_val

Create a adjancency matrix and then create networkx graph

In [10]:
adjancency_df = pd.DataFrame(prob_transitions_mapper)

In [11]:
tmp_dict = adjancency_df.to_dict(orient='records')

In [13]:
weighted_dict = {k1:{k2:{'weight': v2} for k2, v2 in v1.items()} for k1, v1 in enumerate(tmp_dict) }

Using weights now create edgelist graph such that it is fed to node2vec for embedding

In [14]:
G = nx.from_dict_of_dicts(weighted_dict)

In [15]:
nx.write_edgelist(G,'coherence.edgelist',data=['weight'])

**Note**: Need two python files from [node2vec Github Repository](https://github.com/aditya-grover/node2vec/tree/master/src)

Run main file from Node2Vec with setting as below: 
- directed = True
- weighted = True

**After completion of embedding by running main.py file**
Then we read the embedding file

In [16]:
emb_df = pd.read_csv('emb/coherence.emb', sep=' ', skiprows=[0], header=None)
emb_df.set_index(0, inplace=True)
emb_df.reset_index(inplace=True)
emb_df.columns = ['state', 'r', 'g', 'b']
emb_df.sort_values('state', inplace=True)
emb_df.head()

,state,r,g,b
1019,0,0.286288,-0.545620,-0.464838
1013,1,0.189079,-0.431643,-0.578886
1004,2,0.224981,-0.493697,-0.547969
978,3,0.127008,-0.535516,-0.588007
995,4,0.331146,-0.367741,-0.644937


Scaling the values in each column of embedding from 0, 255 for the color codes

In [17]:
t = emb_df.values
i_r = np.interp(t[:, 0], (t[:, 0].min(), t[:, 0].max()), (0, 255))
i_g = np.interp(t[:, 1], (t[:, 1].min(), t[:, 1].max()), (0, 255))
i_b = np.interp(t[:, 2], (t[:, 2].min(), t[:, 2].max()), (0, 255))
i_r = i_r.astype(int)
i_g = i_g.astype(int)
i_b = i_b.astype(int)

Viewing color descriptions:

In [18]:
color_codes_node2Vec = {row_num: {'r':i_r[row_num], 'g':i_g[row_num], 'b':i_b[row_num], 'a': 0} for row_num in range(len(i_r))}

In [19]:
color_node2Vec_dataframe = pd.DataFrame(color_codes_node2Vec).T.drop('a', axis=1)
color_node2Vec_dataframe.describe()

,b,g,r
count,1024.000000,1024.000000,1024.000000
mean,204.174805,118.942383,127.001953
std,25.754759,24.313680,73.743406
min,0.000000,0.000000,0.000000
25%,200.000000,112.000000,63.000000
50%,209.000000,120.000000,127.000000
75%,215.000000,127.000000,191.000000
max,255.000000,255.000000,255.000000


In [20]:
# To store color_codes so that we can view in 3-D plots
color_node2Vec_dataframe.to_csv('color_codesnode2Vec.csv', index=False)

In [21]:
color_codes = color_codes_node2Vec

Note folder named gephi is used to save or can change folder name below

In [22]:
def run_simulation(alpha, coh_matrix, list_agents, end_time):
    
    d = []
    selected_time_stamps = [0, 24, 49, 74, 99, 199, 299]
    for t in range(end_time):    
        # compute next state for all agents
        for agt in list_agents:
            agt.update_knowledge(alpha, coh_matrix) 
         
        # keep record of current record and all other values
        for agt in list_agents:
            row = {'Agent_Name':agt.name,
                   'Agent_Dissonance':agt.dissonance_lst,
                   'Time':t,
                   'Current State':agt.knowledge_state,
                   'Next State':agt.next_state}
            
            d.append(row)
        
        agent_states = {agent.name: utilities.bool2int(agent.knowledge_state) for agent in list_agents}
        if t in selected_time_stamps:
            G=nx.Graph()

            for agent in list_agents:
                agent_num = int(''.join([ch for ch in agent.name if ch.isdigit()]))
                agent_neighbors = [(int(''.join([ch for ch in neighbor_name if ch.isdigit()])), agent_states[neighbor_name]) for neighbor_name in agent.neighbors]
                nghbrs = [int(''.join([ch for ch in neighbor_name if ch.isdigit()])) for neighbor_name in agent.neighbors]
                agent_state = utilities.bool2int(agent.knowledge_state)
                G.add_node(agent_num)
                G.node[agent_num]['viz'] = {'color': color_codes[agent_state]}
                G.node[agent_num]['state'] = agent_state

                for n_agent in nghbrs:
                    G.add_edge(agent_num, n_agent)

            nx.write_gexf(G, "gephi/network_alpha_{}_t_{}.gexf".format(alpha,t))
            
        # now update all agents next state with computed next state
        for agt in list_agents:
            agt.knowledge_state = agt.next_state
            agt.next_state = None
            agt.dissonance_lst = None
            
    return pd.DataFrame(d)

For n number of agent we split each bit to be half 0 and half 1

In [23]:
first_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
second_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
third_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
fourth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
fifth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
sixth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
seventh_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
eigth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
ninth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
tenth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]

shuffle(first_bit)
shuffle(second_bit)
shuffle(third_bit)
shuffle(fourth_bit)
shuffle(fifth_bit)
shuffle(sixth_bit)
shuffle(seventh_bit)
shuffle(eigth_bit)
shuffle(ninth_bit)
shuffle(tenth_bit)

Run simulations for different alpha values and t=0 to t=300

In [24]:
end_simulation_time = 300
alphas = [0,.25,.5,.75,1.0]
    
# first create environment
agents_list = create_environment()
    
# get network of the agents
agent_network_df = get_network_df(agents_list) 
    
results = {}
    
# for saving
agent_network_df.to_json('gephi/test_network.json',orient='records', lines=True)
        
results['seed'] = 1
results['coherence_matrix'] = coherence_matrix
        
    
results['alphas'] = defaultdict(list)
all_df = pd.DataFrame()

In [25]:
# run simulation
for alpha in alphas:
    record_df = None
    record_df = run_simulation(alpha, coherence_matrix, copy.deepcopy(agents_list), end_simulation_time)
    record_df['Current_Knowledge_State'] =  record_df['Current State']
    record_df['Current State'] = record_df['Current State'].apply(lambda row: utilities.bool2int(row))
    record_df['Next State'] = record_df['Next State'].apply(lambda row: utilities.bool2int(row))
    record_df['alpha'] = str(alpha)
    all_df = all_df.append(record_df)
    df = record_df.groupby(['Current State', 'Next State']).size().to_frame('Count').reset_index()

Save knowledge transition graph at t=300 for different alpha values

In [26]:
for a in alphas:
    G = nx.DiGraph()
    x = all_df[all_df['alpha']==str(a)].groupby(['Current State', 'Next State']).size().to_frame('Count')
    x.reset_index(inplace=True)
    for _, row in x.iterrows():
        curr_state = row['Current State']

        G.add_node(curr_state)
        G.node[curr_state]['viz'] = {'color': color_codes[curr_state]}
        G.add_edge(curr_state, row['Next State'], weight=row['Count'])

    nx.write_gexf(G, "knowledge_{}.gexf".format(a))

In [27]:
pd.DataFrame(coherence_matrix).to_csv('gephi/coh_matrix.csv')